## Import

In [116]:
import dagsim.base as ds
import numpy as np
from PIL import Image

In [27]:
import os
print(os.getcwd())

/home/ghadi/Documents/PhD/dagsimManuscript


In [146]:
# https://newbedev.com/how-can-i-create-a-circular-mask-for-a-numpy-array
    
def create_circular_mask(h=256, w=256, center=None, radius=10):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = (dist_from_center <= radius)*256
    return mask
    

def Ber(U1):
    return np.random.binomial(1, 1-U1)

def BerExp(C, Dnum, Dstr):
    if Dstr=="H":
        out = 0.75*Dnum + 0.5*C + 0.25
    else:
        out = 2.5*Dnum + 1.75*C - 0.25
    out = 1/(1 + np.exp(-out))
    out = np.random.binomial(1, out)
    return out

def drawImage(H, V, R, C, output_path):
    image = np.zeros(shape=(256, 256))
    randInd = np.random.randint(low=1, high=10000)
    if H==1:
        randPosH = np.random.randint(low=10, high=246)
#         print(randPosH)
        image[randPosH-5:randPosH+5, :]=256
        
    if V==1:
        randPosV = np.random.randint(low=10, high=246)
#         print(randPosV)
        image[:, randPosV-5:randPosV+5]=256
        
    if C==1:
#         print("cir")
        randC1 = np.random.randint(low=10, high=246)
        randC2 = np.random.randint(low=10, high=246)
        
        mask = create_circular_mask(center=(randC1, randC2))
        image = image + mask
#         circle = (randC1) ** 2 + (randC2) ** 2
#         donut = numpy.logical_and(circle < (5), circle > (5))
        
        
    image = image + np.random.binomial(1, 0.005, size=(256,256))*256
    image = Image.fromarray(image)
    image = image.convert("L")
    image.save(output_path + "/" + str(randInd) + ".png")

In [147]:
U1 = ds.Generic(name="U_1", function=np.random.uniform)
U2 = ds.Generic(name="U_2", function=np.random.uniform)

H = ds.Generic(name="H", function=np.random.binomial, arguments={"n":1, "p":U1})
C = ds.Generic(name="C", function=np.random.binomial, arguments={"n":1, "p":U2})

V = ds.Generic(name="V", function=Ber, arguments={"U1":U1})

R = ds.Generic(name="R", function=BerExp, arguments={"C":C, "Dnum":H, "Dstr":"H"})
Y = ds.Generic(name="Y", function=BerExp, arguments={"C":C, "Dnum":V, "Dstr":"V"})

I = ds.Generic(name="Image", function=drawImage, arguments={"H":H, "V":V, "R":R, "C":C}, visible=False)

graph = ds.Graph(name="testImages", list_nodes=[U1, U2, H, V, C, R, Y, I])
# graph.draw()

In [150]:
folder = './images'
if os.path.exists(folder):
    for f in os.listdir(folder):
        os.remove(os.path.join(folder, f))
else:
    os.mkdir(folder)
    
#     shutil.rmtree('./images')
# os.mkdir("./images")

n = graph.simulate(num_samples=50, output_path="./images")

Simulation started
Simulation finished in 0.1041 seconds


## ----------------------------------------------------------------------------------------------------------------